In [1]:
import cv2
import numpy as np
import scipy as sp

import matplotlib as mpl
import matplotlib.pyplot as plt

import os
import errno

from os import path
from glob import glob

from scipy.spatial import distance
from scipy import ndimage

# Use jupyter "magic" methods to dynamically reload external modules every
# time any block is run, and show images inline in the notebook
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [235]:
# Change the source folder and exposure times to match your own
# input images. Note that the response curve is calculated from
# a random sampling of the pixels in the image, so there may be
# variation in the output even for the example exposure stack
SRC_FOLDER = "images/source/original1"
OUT_FOLDER = "images/output"
EXTENSIONS = set(["bmp", "jpeg", "jpg", "png", "tif", "tiff"])

In [236]:
def generate_animation_frames(left_image, right_image, camera_angle, fov, distance_factor, num_int_frames, output_folder):
    """ Generates intermediate frames that will be used to smooth out the transition between our two 
        stereo images in order to give a more credible 3d rotation or 'wiggle effect'.
        
        Parameters
        ----------
        left_image: np.ndarray
            First stereo image 
        right_image: np.ndarray
            Second stereo image
        camera_angle: int
            Angle between left and right camera
        fov: <int>
            Field of view of camera
        distance_factor: int
            Distance parameter for better depth perception
        num_int_frames: int
            Number of intermediate frames we want to generate
        output_folder: string
            Directory where the resulting frames will be saved
            
        Returns
        ----------
        frames: list<np.ndarray>
            The set of resulting intermediate frames generated
    """
    # Initialize output array
    frames = []

    # Generate angles for all intermediate frames
    int_angles = np.linspace(0, camera_angle, num=num_int_frames, endpoint=False, dtype=np.float64)
    
    # To generate an infinite wiggle image duplicate intermediate frames
    tot_frames = num_int_frames * 2
        
    # Generate intermediate frames and save output
    for i in range(1,len(int_angles)):
        # Extract intermediate image angle
        int_cam_angle = int_angles[i]
        
        print "Generating frame: '" + str(i) + " with angle " + str(int_cam_angle) + " ..."
        
        # Generate intermediate frame
        out = generate_intermediate_frame(left_image,right_image,camera_angle,int_cam_angle, fov, distance_factor)
        
        # Save the frame 
        cv2.imwrite(path.join(output_folder, "frame" + "%03d" % i + ".png"), out)

        # Save Duplicate frame to create gif
        cv2.imwrite(path.join(output_folder, "frame" + "%03d" % (tot_frames - i) + ".png"), out)
            
        # Append result to frames list
        frames.append(out)
    
    return frames

In [287]:
def main(image_files, output_folder, resize=False):
    """ This function takes care of Setting up the input images in the direactory. 
        It is where camera and image parameters are adjusted. It also takes care 
        or saving frames and generating the resulting animation of our algorithm.
        
        Parameters
        ----------
        image_files: 
        
        output_folder:
        
        resize(False):
        
        Returns
        ----------
        Nothing
    """
    img_stack = [cv2.imread(name) for name in image_files
                 if path.splitext(name)[-1][1:].lower() in EXTENSIONS]

    if any([im is None for im in img_stack]):
        raise RuntimeError("One or more input files failed to load.")

    # Subsampling the images can reduce runtime for large files
    if resize:
        img_stack = [img[::4, ::4] for img in img_stack]
    
    left_image = img_stack[0]
    right_image = img_stack[1]
    
    # Parameters
    camera_angle = 1.9#3.5#1.9
    fov = 100
    num_int_frames = 6
    distance_factor = 1
    
    frames = generate_animation_frames(left_image, right_image, camera_angle, 
                                       fov, distance_factor, num_int_frames, output_folder)
    
    # Save initial and final frames to output directory
    cv2.imwrite(path.join(output_folder, "frame" + "%03d" % 0 + ".png"), left_image)
    cv2.imwrite(path.join(output_folder, "frame" + "%03d" % (len(frames)+1) + ".png"), right_image)
    
    # Convert frames to gif
    command = "ffmpeg -i " + path.join(output_folder, "frame%03d.png") + " -r 10 " + path.join(output_folder, "out_video.gif")    
    os.system(command)


    print "Done!"

In [288]:
def run():
    """ Main function that takes care of reading through directory and running main 
        function with our read stereo image pair.
        
        Parameters
        ----------
        None
        
        Returns
        ----------
        Nothing
    """
    src_contents = os.walk(SRC_FOLDER)
    dirpath, _, fnames = src_contents.next()

    image_dir = os.path.split(dirpath)[-1]
    output_dir = os.path.join(OUT_FOLDER, image_dir)
    print(output_dir)
    try:
        os.makedirs(output_dir)
    except OSError as exception:
        if exception.errno != errno.EEXIST:
            raise

    print "Processing '" + image_dir + "' folder..."

    image_files = sorted([os.path.join(dirpath, name) for name in fnames])
    
    main(image_files, output_dir, resize=False)

# RUN ME

In [290]:
run()

images/output/original4
Processing 'original4' folder...
Generating frame: '1 with angle 0.316666666667 ...
('Pivot Distance: ', 255.0)
('Number of holes', 1383)
('Final Number of holes', 0)
Generating frame: '2 with angle 0.633333333333 ...
('Pivot Distance: ', 255.0)
('Number of holes', 1458)
('Final Number of holes', 0)
Generating frame: '3 with angle 0.95 ...
('Pivot Distance: ', 255.0)
('Number of holes', 1461)
('Final Number of holes', 0)
Generating frame: '4 with angle 1.26666666667 ...
('Pivot Distance: ', 255.0)
('Number of holes', 1407)
('Final Number of holes', 0)
Generating frame: '5 with angle 1.58333333333 ...
('Pivot Distance: ', 255.0)
('Number of holes', 1416)
('Final Number of holes', 0)
ffmpeg -i images/output/original4/frame%03d.png -r 10 images/output/original4/out_video.gif
Done!


### Intermediate Frame Generation

In [260]:
def generate_intermediate_frame(image_L,image_R,camera_angle,int_camera_angle,fov,distance_factor):
    """Computational pipeline to produce an intermediate image using a pivot point from a depth map
    of a set of stereoscopic images.

    The basic overview is to do the following for each pair of stereo images:

    1. Generate a depth map
       
    2. Select a pivot point

    3. Use pivot point as guide to shift the image into new orientation

    4. Map pixel values to new orientation and return the intermediate image

    Parameters
    ----------
    image_L : <numpy.ndarray>
        The first stereo image
    image_R : <numpy.ndarray>
        The second stereo image
    camera_angle: <int>
        Angle between left and right camera
    int_camera_angle: <int>
        Angle of intermediate image from the left image
    fov: <int>
        Field of view of camera
    distance_factor: <int>
        Distance parameter for better depth perception
        
    Returns
    -------
    numpy.ndarray
        The resulting intermediate image generated
    """
    # Extract depth map
    depth_map = cv2.imread(path.join(SRC_FOLDER, "depth/depth_map.png"), cv2.CV_LOAD_IMAGE_GRAYSCALE)
    depth_map = np.multiply(depth_map.astype(np.float64)-255, -1)
#     depth_map = calc_depth_map(image_L, image_R) # Automatic
    
    # Select Pivot Distance
    y = set(depth_map.flatten())
    pivot_dist = (max(y) + min(y)) /2 #ndimage.median(depth_map) # median of all distance values
    
    # Shift depth_map values
    depth_map = depth_map + pivot_dist*distance_factor
    pivot_dist = pivot_dist + pivot_dist*distance_factor
    print("Pivot Distance: ",pivot_dist)
    
    # Save images into array
    image_list = [image_L, image_R]

    # Matrices that (for each image) holds information on where to send each pixel for the intermediate image
    dtype = [('image_idx', int), ('angle', float)]
    values = [(0, int_camera_angle),(1, -(camera_angle-int_camera_angle))]
    image_data = np.array(values, dtype=dtype)
    
    # Using the int camera angle, determine which camera is closest and use its image
    image_data = np.sort(image_data,order='angle')
    
    # Extract image dimensions
    rows, cols, channel = image_L.shape

    # Initialize transport maps
    transport_maps = np.zeros((2,rows,cols,2))

    # Initialize output image
    int_img = np.zeros((rows, cols, channel)) + np.nan
    
    # Using the depth map, order the pixels from furthest to closest
    dtype = [('row', int), ('col', int), ('depth', np.int16)]
    dr = np.arange(rows)
    dc = np.arange(cols)
    depth_map_dict = np.array(np.zeros(depth_map.shape), dtype=dtype)
    depth_map_dict['row'] = dr[:,None]
    depth_map_dict['col'] = dc
    depth_map_dict['depth'] = depth_map
    sorted_dm = np.sort(depth_map_dict.flatten(), order='depth')[::-1].reshape(rows,cols)
    
    # Generate Transport Maps
    for i in range(2):
        # Extract Values
        image = image_list[image_data[i]['image_idx']]
        int_cam_angle = image_data[i]['angle']
        transport_map = transport_maps[i]
        for k in range(rows): # Y values
            for j in range(cols): # X values
                (row,col) = (sorted_dm[k,j][0], sorted_dm[k,j][1])
                
                # Change pixels to FOV degrees
                pixel_fov_deg = (float(col)/float(cols))*fov - (fov/2)
                pixel_fov_rad = np.deg2rad(pixel_fov_deg)
                pixel_dist = sorted_dm[k,j][2]
                
                # Calculate pixel angle and radius with pivot
                r,theta_p = pivot_ang_rad(pixel_fov_rad,pivot_dist,pixel_dist)
                
                # Calculate pixel fov angle
                int_fov_deg = adjusted_fov(r,theta_p,int_cam_angle, pivot_dist)
                int_fov_deg = np.rad2deg(int_fov_deg) # Convert back to degrees
                
                # Convert intermediate field-of-view degrees back into pixel location values
                x_pixel = (int_fov_deg+(fov/2))*cols/fov                
                x_pixel = int(round(x_pixel))

                # Check if x_pixel is within bounds 
                if not np.isnan(x_pixel) and x_pixel < cols and x_pixel > 0:
                    transport_map[row,col] = [row, x_pixel]
                    
                    # Fill the black spots with the pixels of current stereo image
                    int_img[row,x_pixel] = image[row,col]
    
    int_img = patch_holes(int_img)
        
    return int_img.astype(np.uint8)

## Helper Functions

### 1.1 

In [185]:
def pivot_ang_rad(pixel_ang,pivot_dist,pixel_dist):
    """Calculates a pixels distance from the pivot and its angle of rotation around it

    Parameters
    ----------
    # pixel_ang:
        angle (radians) between pixel and pivot from camera
    # pivot_dist:
        distance from camera to pivot
    # pixel_dist:
        distance from camera to pixel
        
    Returns
    -------
    # r: distance from pivot
    # angle: angle of rotation
    """
    # Pixel at pivot angle case
    if (pixel_ang == 0):
        dist = pivot_dist - pixel_dist
        # Pixel is in front of pivot
        if (dist >= 0):
            angle =  np.deg2rad(0)
        # Pixel is behind the pivot
        else:
            angle = np.deg2rad(180)
        r = abs(dist)
        return(r,angle)
    
    # Normal Case
    else:
        x = pixel_dist*np.sin(pixel_ang) 
        y = pixel_dist*np.cos(pixel_ang)
        z = pivot_dist - y
        
        # Case where inv tan can't be computed
        if (z == 0):
            r = abs(x)
            if (x > 0):
                angle = np.deg2rad(90)
            else:
                angle = np.deg2rad(-90)
            return(r,angle)
        
        # Case where inv tan can be computed
        angle = np.arctan(x/z)
        r = np.sqrt(x**2 + z**2)
        if (z < 0):
            angle = angle + np.deg2rad(180)
        return(r.astype(np.float64),angle.astype(np.float64))

### 1.2

In [186]:
def adjusted_fov(r,pixel_ang,int_cam_angle, pivot_dist):
    """ Calculates FOV angle for pixel in the intermediate camera

    Parameters
    ----------
    # r: 
        distance from pixel to pivot point
    # angle:
        angle the pixel is rotated from center in the camera
        
    Returns
    -------
    # angle: FOV angle
    """
    
    #Convert int_cam_angle to Rad
    int_cam_angle = np.deg2rad(int_cam_angle)
    
    # Calculate relative rotation of pixel for intermediate camera's pov
    i_angle = pixel_ang - int_cam_angle
    
    # Case where new pixel rotation is right in front or behind pivot axis
    if (i_angle % np.deg2rad(180) == 0):
        angle = 0.0
        return(angle)
        
    # Normal Case
    x = r * np.sin(i_angle)
    y = r * np.cos(i_angle)
    z = pivot_dist - y
    
    # Arctan will always evaluate since z != 0
    angle = np.arctan(x/z)
    return(angle)

### 1.3

In [1]:
def patch_holes(image):
    """ Helper function that takes care of interpolating pixel values that were not
        able to be assigned during our remapping algorithm for the intermediate image.
        
        Parameter:
        ----------
        image: numpy.ndarray
        
        Returns:
        ----------
        out: numpy.ndarray
        Patched image with interpolated pixel values in 'holes' or unasigned pixels.
    """
    # Extract image dimensions
    rows, cols, channels = image.shape
    
    # Check for any left over holes and if there are any fill in remaining pixels
#     print("Number of holes", np.count_nonzero(np.isnan(image)))
    
    # If there are remaining nan values, replace them by interpolating amongst neighbors
    if (np.count_nonzero(np.isnan(image)) > 0):
         for i in range(rows): # Y values
            for j in range(cols): # X values
                # If this pixel is nan, interpolate from neighbors to replace value
                if(np.isnan(image[i,j]).all()):
                    # Find closest indexes to pixel that is not nan
                    left_cols = image[i,0:j]
                    right_cols = image[i,j+1:cols]
                    
                    # Make sure l_idx and r_ix are valid values
                    try:
                        l_idx = np.nanargmax(left_cols[:,0])
                        l_w = float(1.0)
                    except:
                        left_cols = np.zeros((1,channels))
                        l_idx = 0
                        l_w = float(0.0)
                    try:
                        r_idx = np.nanargmin(right_cols[:,0])
                        r_w = float(1.0)
                    except:
                        right_cols = np.zeros((1,channels))
                        r_idx = 0
                        r_w = float(0.0)
                    
                    # Fill spot using neighboring columns
                    image[i,j] = [np.average([left_cols[l_idx,x], right_cols[r_idx,x]], weights=[l_w, r_w]) for x in range(channels)]
    
#     print("Final Number of holes", np.count_nonzero(np.isnan(image)))
    
    return image

## Map Calculations

### 1.1 Optical Flow Map

In [13]:
def calc_optical_flow(img1, img2):
    """ Calculates optical flow from a pair of stereo images.
    
    Parameters
    ----------
    # img_l: <np.ndarray>
        First stereo image
        
    # img_r: <np.ndarray>
        Second stereo image
        
    Returns
    -------
    optical flow map: <numpy.ndarray>
         
    """
    # Initialization
    hsv = np.zeros_like(img1)
    # Convert images to single channel by making grayscale
    gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
    
    
    # while(1):
    # Gunnar Farnebäck. Two-frame motion estimation based on polynomial expansion. In Image Analysis, pages 363–370. Springer, 2003.
    # img1, img2, flow, pyr_scale, levels, winsize, iterations, poly_n, poly_sigma, flags
    # defaults: flow=None, pyr_scale=0.5, poly_n=5 or 7, poly_n=5, use poly_sigma=1.1, poly_n=7, use poly_sigma=1.5
    flow = cv2.calcOpticalFlowFarneback(gray1,gray2, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    # show hsv as bgr color image
    hsv[...,1] = 255
    mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
    hsv[...,0] = ang*180/np.pi/2
    hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    bgr = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)

    return bgr

### 1.2 Depth Map

In [247]:
def calc_depth_map(img_l, img_r):
    """ Calculates depth map from a set of stereo images.
    
    Parameters
    ----------
    # img_l: <np.ndarray>
        First stereo image
        
    # img_r: <np.ndarray>
        Second stereo image
        
    Returns
    -------
    depth_map: <numpy.ndarray>
        Grayscale image with depth values at each pixel
    """
    # Get output directory
    src_contents = os.walk(SRC_FOLDER)
    dirpath, _, fnames = src_contents.next()

    image_dir = os.path.split(dirpath)[-1]
    output_dir = os.path.join(OUT_FOLDER, image_dir)
    
    # Convert images to grayscale
    gray1 = cv2.cvtColor(img_l,cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(img_r,cv2.COLOR_BGR2GRAY)
    
    # Blur the images
    gray1 = cv2.GaussianBlur(gray1,(5,5),0)
    gray2 = cv2.GaussianBlur(gray2,(5,5),0)
    
    # Calculate disparity 
    #ndisparities should be divisible by 16, SADWindowSize should be odd
    stereo = cv2.StereoBM(cv2.STEREO_BM_BASIC_PRESET,ndisparities=16, SADWindowSize=51)
    disparity = stereo.compute(gray1,gray2).astype(np.float64) # 16, 101
    
    if np.min(disparity) < 0:
        # Shift disparity to be centered
        disparity = np.add(disparity, abs(np.min(disparity)))
        
    # Normalize results
    out = cv2.normalize(disparity, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)
    
    # Save results
    cv2.imwrite(path.join(output_dir, "depth_map.png"), out)
    
    # Adjust Values
    return np.multiply(out-255, -1)

## Energy Computations

In [8]:
# Energy Computations
""" """

# Depth Energy
def calc_depth_energy(depth_map):
    """ Will be calculated based on a preference for position the pivot point on regions
        close to the middle of the depth map

    Depth Energy: We use the depth energy to express a preference
    for positioning the pivot point on regions that are
    close to the middle of the depth map. This minimizes the
    visibility of the occluded regions during the rendering camera
    movements and also lowers the amount of motion when
    the final animation is generated. We obtain the depth map
    of a given image set by calculating its optical flow using
    Sun et al.’s [48] algorithm. Details about depth map calculation
    are described in Section 8. After calculating the
    depth map we can calculate the depth energy as: 
    Ed(x, y) = ||Pd(x, y)−Dm||2
        where Pd(x, y) refers to the depth value of the pixel at (x, y) 
        and Dm is the median depth of the scene.
    """
    median_depth = ndimage.median(depth_map)

    depth_energy = np.sqrt((depth_map - median_depth)**2)

    return depth_energy

In [9]:
# Saliency Energy
def calc_saliency_energy(img):
    """ """
    # Change input to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Initialize saliency algorithm

    saliency = cv2.saliency.StaticSaliencyFineGrained_create()
    saliency.setImagesize(img.shape)
    saliency.init()
    
    # Run algorithm
    out, saliencyMap = saliency.computeSaliency(gray)
    cv2.imwrite(path.join(OUT_FOLDER, "source/saliency_energy.png"), (1 - out))

    return (1 - out)

In [10]:
# Radial Energy
def calc_radial_energy(img):
    """ """
    # Calculate center of image
    cx, cy = img.shape // 2

    # Generate Coordinate vectors
    nx, ny = img.shape
    x = np.arange(nx)
    y = np.arange(ny)
    xcoords, ycoords = np.meshgrid(x, y)

    # Calculate Radial Energy for each pixel
    # E(x,y) = sqrt((cx-x)^2 + (cy-y)^2)
    radial_energy = np.sqrt((cx - xcoords)**2 + (cy - ycoords)**2)

    return radial_energy

## Testing Code Out